In [116]:
import pandas as pd
from pprint import pprint
import requests
import smtplib
import os
from datetime import datetime
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()

# Get environment variables
weather_api_key = os.getenv("weather_api_key")
my_email = os.getenv('adventure_email')
password = os.getenv('adventure_password')
verify = os.getenv('adventure_twostep_pass')
lat = os.getenv('SCLAT')
long = os.getenv('SCLONG')

OWM_Endpoint = "https://api.openweathermap.org/data/3.0/onecall"
parametres = {
    #lat and long
    "lat": lat,
    "lon": long,
    "exclude": "current,minutely,daily,alerts",
    "appid": weather_api_key,
}

response = requests.get(url=OWM_Endpoint, params=parametres)
response.raise_for_status()
#print(response.status_code)
weather_data = response.json()

In [112]:
pprint(weather_data)

{'hourly': [{'clouds': 68,
             'dew_point': 275.99,
             'dt': 1738303200,
             'feels_like': 278.46,
             'humidity': 80,
             'pop': 0,
             'pressure': 1025,
             'temp': 279.17,
             'uvi': 0,
             'visibility': 10000,
             'weather': [{'description': 'broken clouds',
                          'icon': '04n',
                          'id': 803,
                          'main': 'Clouds'}],
             'wind_deg': 302,
             'wind_gust': 1.72,
             'wind_speed': 1.35},
            {'clouds': 74,
             'dew_point': 276.31,
             'dt': 1738306800,
             'feels_like': 279.68,
             'humidity': 79,
             'pop': 0,
             'pressure': 1025,
             'temp': 279.68,
             'uvi': 0,
             'visibility': 10000,
             'weather': [{'description': 'broken clouds',
                          'icon': '04n',
                          'id':

In [113]:
pprint(weather_data['hourly'][14])

{'clouds': 100,
 'dew_point': 279.3,
 'dt': 1738353600,
 'feels_like': 282.52,
 'humidity': 79,
 'pop': 1,
 'pressure': 1026,
 'rain': {'1h': 0.49},
 'temp': 283.38,
 'uvi': 0.58,
 'visibility': 6075,
 'weather': [{'description': 'light rain',
              'icon': '10d',
              'id': 500,
              'main': 'Rain'}],
 'wind_deg': 190,
 'wind_gust': 2.14,
 'wind_speed': 1.06}


In [114]:
weather_slice_12_hours = weather_data['hourly'][1:47]

daily_forecast_dict = {
    "time":[],
    'date':[],
    "weekday":[],
    'main_weather':[],
    'weather_description':[],
    'rain':[],
    "temp_c":[]
}

for hour_slice in weather_slice_12_hours:
    date_time = hour_slice['dt']
    daily_forecast_dict['date'].append(datetime.utcfromtimestamp(date_time).strftime('%Y-%m-%d'))
    daily_forecast_dict['time'].append(datetime.utcfromtimestamp(date_time).strftime('%H:%M:%S'))
    daily_forecast_dict['weekday'].append(datetime.utcfromtimestamp(date_time).strftime("%A"))
    daily_forecast_dict['main_weather'].append(hour_slice['weather'][0]['main'])
    daily_forecast_dict['weather_description'].append(hour_slice['weather'][0]['description'])
    temp_k = hour_slice['temp']
    daily_forecast_dict['temp_c'].append(round(temp_k - 273.15, 1))
    try:
        if hour_slice['rain']:
            daily_forecast_dict['rain'].append(f'{hour_slice["rain"]["1h"]}mm')
  
    except KeyError:
        daily_forecast_dict['rain'].append(0)

daily_forecast_df = pd.DataFrame.from_dict(daily_forecast_dict)
# daily_forecast_df

In [101]:
# main_weather = weather_data['hourly'][24]['weather'][0]['main']
# weather_description = weather_data['hourly'][24]['weather'][0]['description']
# date_time = weather_data['hourly'][24]['dt']
# date = datetime.utcfromtimestamp(date_time).strftime('%Y-%m-%d')
# time = datetime.utcfromtimestamp(date_time).strftime('%H:%M:%S')
# weekday = datetime.utcfromtimestamp(date_time).strftime("%A")
# temp_k = weather_data['hourly'][24]['temp']
# temp_c = round(temp_k - 273.15, 1)

# print(f'On {weekday}, {date} you can expect {main_weather} with a temperature of {temp_c}°C')

# print(datetime.utcfromtimestamp(date_time).strftime('%Y-%m-%d %H:%M:%S'))

In [115]:
will_rain = False
        
if "Rain" in daily_forecast_dict['main_weather']:
    will_rain = True

if will_rain:
    print("Bring an umbrella!")
    with smtplib.SMTP("smtp.gmail.com", port=587) as connection:
        connection.starttls()
        connection.login(user=my_email, password=verify)
        connection.sendmail(
            from_addr=my_email,
            to_addrs=my_email,
            msg=f'Subject: Weather alert!\n\n'
                f'On {weekday}, {date} you can expect {main_weather} with a temperature of {temp_c}C.\n\n'
                f'Bring an Umbrella and maybe take the bus!!\n\n'
                f'{daily_forecast_df}'
        )
else:
    print("no rain!")

Bring an umbrella!
